## Introduction

Subarray observations for the ACS/WFC allow users to read out only selected areas of the CCDs. This can be useful, for example, when using certain filters that heavily vignette the field of view or when the target subtends only a small area on the sky. Subarray frames have a faster readout time compared to the full-frame as well as a smaller data volume, which allows users to pack many more exposures in a single orbit and reduce the number of buffer dumps.

To download this page as a Python 3 Jupyter Notebook, please visit the [acs-notebook GitHub page](https://github.com/spacetelescope/acs-notebook).

### Subarray Definitions

A comprehensive summary of the subarray readout configuations can be found in [ACS ISR 2017-03](http://www.stsci.edu/hst/acs/documents/isrs/isr1703.pdf). For subarray data taken prior to Servicing Mission 4 (SM4), the 2-D bias and dark images are cut out of the full-frame reference files during calibration using `calacs`. However, after SM4 the clock speed of the subarray readout was different than the full-frame, resulting in the need to acquire bias images in each subarray mode to match observations. The subarray definitions in the flight software were updated at the start of Cycle 24 to match the clock speed of the readout, which solves the need for separate bias imaging. Most noticeably, the new subarray definitions allow users to reduce the number of rows in the image, but the number of columns will always be 2072 (2048 + 24 bias prescan columns). This is markedly different from the pre-Cycle 24 format in which both the number of columns and rows were reduced. For example, the old 512 subarray, which measured 512 rows x 534 columns in the raw image, is now 512 x 2072. In addition, the post-Cycle 24 2K subarray also includes 20 rows of virtual overscan to match the full-frame readout.

### Differences in Calibration Steps for Subarray Data

The installation of the Application-Specific Integrated Circuit (ASIC) during SM4 imparted a low-frequency, row-correlated noise component to all post-SM4 ACS/WFC observations ("striping"; [ACS ISR 2011-05](http://www.stsci.edu/hst/acs/documents/isrs/isr1105.pdf)). Full-frame images are corrected for this striping effect as part of the `ACSCCD` step in `CALACS` using the bias prescan areas adjacent to each readout amplifier. This correction leverages the fact that the striping is continuous across amplifier regions to measure the striping level on the unexposed prescan pixels. As only one bias prescan region is available with subarray observations, stripe removal is not performed for subarray data. For subarray data taken prior to Cycle 24, users may choose to apply the striping correction to 2K subarrays, which will use the single bias prescan and sky-exposed pixels on the far side of the image opposite the prescan to estimate the striping level, however this is scene dependent and may produce unsatisfactory results (see the documentation for [acs_destripe_plus](http://acstools.readthedocs.io/en/stable/acs_destripe_plus.html) and Example 1 below). All subarray data taken using the new post-Cycle 24 subarray defintions may use this de-striping technique with the same caveat regarding scene dependence.

[Charge transfer efficiency (CTE)]() loss is typically corrected in the ACS data pipeline by the pixel-based CTE code `ACSCTE`, which is controlled by the `PCTECORR` calibration switch in FITS file primary headers and is configured by the `PCTETAB` reference file. Due to concerns about amplifying the striping noise that is not removed in subarray data, these data are not automatically corrected for CTE loss by `CALACS`. Users are given the choice to apply the CTE correction to subarray data using the `acs_destripe_plus` code.

### Custom Subarray Data

<div class="alert alert-info">In an effort to provide the best-calibrated data possible in the archive, all custom subarray data have been calibrated by the ACS Team and future automated reprocessing has been disabled. Please see the [Status of ACS Data in MAST]() page for information regarding impacted data.</div>

User-defined (or "custom") subarray data were an available-but-unsupported mode prior to SM4 (and then a restricted mode until Cycle 24 when their use was discontinued), however the calibration of these data is complicated within `CALACS`. The `CALACS` data pipeline utilizes the `OSCNTAB` reference file to define the physcial bias prescan and virtual bias overscan regions. During the `BLEVCORR` step in `ACSCCD`, the physical prescan regions are used to subtract the bias level from each row, and then the prescan and overscan regions are trimmed from the image. Because custom subarray data can include any or none of the prescan columns, they are not defined in `OSCNTAB`. We show how to update the table with a new definition for a custom subarray observation in the example below.

The subtraction of the 2-D bias ("superbias") image from custom subarray data is also complicated by two factors:

1. Pre-SM4: If the data do not contain any bias prescan columns, the `BLEVCORR` step will be skipped and the bias level will be entirely removed using the superbias image, which must be made from bias frames taken as part of the observing program to include the full bias level. This is in contrast to subarray data with bias prescan columns that use a cutout of the superbias image from the full-frame reference file.

2. Post-SM4: As discussed above, differences in the clocking rate between subarray and full-frame observations taken after SM4 require subarray data to have corresponding bias frames taken with the same subarray readout definitions to create the superbias image.

The [CRDS](https://hst-crds.stsci.edu) is unable to match appropriately sized superbias reference files to custom subarray images. In cases where such a reference file is required, users will need to either create it themselves or contact the ACS Team via the [help desk](http://hsthelp.stsci.edu) for assistance.

<div class="alert alert-warning">Before running these examples, be sure to update you [AstroConda](https://astroconda.readthedocs.io/en/latest/) distribution to the lastest version. You will also need to install `astroquery` via:
<br><br>
```
conda install astroquery
```
<br><br>
which will allow you to download the datasets used in these examples.</div>

## Examples

### Example 1: De-striping and CTE Correction of Post-SM4 Subarray Observations

Our example data come from an observation of the globular cluster 47 Tuc (NGC 104). We will use the `astroquery` tool to download the dataset. The association name is JD5702011, and it is comprised of two images using the post-Cycle 24 WFC1A-1K 1024 x 2072 subarray taken as part of the ACS CCD Stability Monitor routine calibration program (*HST* proposal 14511). The example will only use the first image in the association with the rootname JD5702JMQ.

#### Step 1: Download the Data

In [ ]:
from astroquery.mast import Observations
import glob
import shutil

obs_table = Observations.query_criteria(obs_id='JD5702011')

Observations.download_products(obs_table['obsid'], mrp_only=False,
                              productSubGroupDescription=['RAW', 'SPT'])

files = glob.glob('mastDownload/HST/*JMQ/*.fits')
for f in files:
    shutil.move(f, '.')
    
shutil.rmtree('mastDownload')

#### Step 2: Update Header Keywords

Next we will need to turn on the CTE correction switch in the primary header and add the names of the `PCTETAB` and `DRKCFILE` reference files to the header. These are the CTE correction parameters table and the CTE-corrected dark current image, respectively. This information can be retrieved using the [CRDS](https://hst-crds.stsci.edu) `bestrefs` tool.

In [ ]:
from astropy.io import fits
import os

with fits.open('jd5702jmq_raw.fits', mode='update') as hdu:
    hdu[0].header['PCTECORR'] = 'PERFORM'

os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = './crds_cache'
os.environ['JREF'] = './crds_cache'

!crds bestrefs --files 'jd5702jmq_raw.fits' --sync-references=1 --update-bestrefs

#### Step 3: Run acs_destripe_plus

Next we will run the [acs_destripe_plus](http://acstools.readthedocs.io/en/stable/acs_destripe_plus.html) code on our image. This will execute all of the calibration steps that are set to 'PERFORM' in the primary header of the FITS file. Note that with the CTE correction turned on, this can take some time. On an 8 core machine, the CTE correction will take approximately 3 minutes to complete while using all of the cores. The `acs_destripe_plus` code will produce two new files: jd5702jmq_flt.fits and jd5702jmq_flc.fits. The FLT file is the calibrated output product from `CALACS`, while the FLC file is the FLT image with the CTE correction applied.

In [ ]:
from acstools import acs_destripe_plus

acs_destripe_plus.destripe_plus('jd5702jmq_raw.fits', cte_correct=True)

#### Step 4: Correct the WCS

The subarray products produced by this process do not have the proper WCS information in the header. The WCS is normally updated by the pipeline via an additional call to AstroDrizzle. Here, we can manually update the WCS of our FLC product using `stwcs.updatewcs`.

In [ ]:
from stwcs import updatewcs

updatewcs.updatewcs('jd5702jmq_flc.fits', use_db=False)

### Example 2: Bias Prescan Subtraction of Custom Subarray Data

Our example data come from a ramp filter observation of the galaxy PLS1345+12 (*HST* proposal 10206). The association name is J92SA0010, and we will only use one image in the association: J92SA0W6Q. The raw image has 1180 columns and 1200 rows, which does not correspond to any entry in the `OSCNTAB` file, but a visual examination of the image shows that it contains bias prescan columns.

The [acstools](http://acstools.readthedocs.io) package has been updated to include a tool for checking the `OSCNTAB` file if any entry matches the combination of readout amplifier, image size, and number of bias prescan columns for a given subarray observation. We will need to use several header keyword values to test if a subarray is in `OSCNTAB` and, if not, we will update the reference file appropriately.

#### Step 1: Download the Data

In [ ]:
from astroquery.mast import Observations
import glob
import shutil

obs_table = Observations.query_criteria(obs_id='J92SA0010')

Observations.download_products(obs_table['obsid'], mrp_only=False,
                               productSubGroupDescription=['RAW', 'SPT'])

files = glob.glob('mastDownload/HST/*w6q/*.fits')
for f in files:
    shutil.move(f, '.')
    
#shutil.rmtree('mastDownload')

We also want to retrieve the `OSCNTAB` reference file from the JREF directory. We can get the name of the file from the primary header of the image.

In [ ]:
from astropy.io import fits
import os

# The name in the header is of the format jref$oscn_name.fits,
# therefore we need to split the string on the '$' character.
with fits.open('j92sa0w6q_raw.fits', mode='update') as hdu:
    prihdr = hdu[0].header
    scihdr = hdu[1].header
    
    prihdr['DARKCORR'] = 'OMIT'
    prihdr['BIASCORR'] = 'OMIT'
    prihdr['SINKCORR'] = 'OMIT'
    prihdr['FLATCORR'] = 'OMIT'
    prihdr['PHOTCORR'] = 'OMIT'
    
os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = './crds_cache'
os.environ['JREF'] = './crds_cache'

!crds bestrefs --files 'j92sa0w6q_raw.fits' --sync-references=1 --update-bestrefs

oscn_name = prihdr['OSCNTAB'].split('$')[-1]

# Now we join the file name with the expanded JREF environment 
# variable.
path_to_oscn = os.path.join(os.environ['jref'], oscn_name)
shutil.copy(path_to_oscn, './my_oscntab.fits');

#### Step 2: Get the Bias Prescan Information

In [ ]:
from acstools import utils_calib

with fits.open('j92sa0w6q_spt.fits') as hdu:
    spthdr = hdu[1].header

# From the science image primary and SCI extension headers and the SPT
# first extension header, we can get the information about the readout
# amplifier, dimensions of the image, and number of prescan columns if
# any.
amp = prihdr['CCDAMP']
xsize = scihdr['NAXIS1']
ysize = scihdr['NAXIS2']
leading = spthdr['OVERSCNA']
trailing = spthdr['OVERSCNB']

# Finally, we check if this subarray definition is in the OSCNTAB
# file. The code returns a boolean result, which we have saved
# as the variable supported, to describe this.
supported = utils_calib.check_oscntab('my_oscntab.fits', amp, xsize, ysize, 
                                      leading, trailing)

print('Subarray Definition')
print('-------------------')
print('Readout Amplifier: {}'.format(amp))
print('X-size: {} pixels'.format(xsize))
print('Y-size: {} pixels'.format(ysize))
print('Leading prescan columns: {}'.format(leading))
print('Trailing prescan columns: {}'.format(trailing))
print('Defined in OSCNTAB? {}'.format(supported))

#### Step 3: Update OSCNTAB

Now that we have confirmed that the `OSCNTAB` file does not contain information about our subarray data, we need to add a new row to the table with our definitions. Let's first view the first few rows of `OSCNTAB` to see what our new entry needs to look like.

In [ ]:
from astropy.table import Table

with fits.open('my_oscntab.fits', mode='update') as hdu:
    oscntab = hdu[1].data
    
tab = Table(oscntab)
tab[:3]

Several column names are obvious, but here we define the less obvious ones. 

* **BINX, BINY:** Binning in X and Y. ACS data are never binned, so these will always be 1.
* **TRIMXn:** Number of prescan columns on the left (1) and right (2) sides of the image to remove.
* **TRIMYn:** Number of virtual rows on the bottom (1) and top (2) sides of the image to remove. For subarray data, these are always 0.
* **BIASSECTAn, BIASSECTBn:** Start and end columns to use for the bias level estimation on the left (A) and right (B) sides of the image.
* **VXn, VYn:** The coordiantes (VX1, VY1), (VX2, VY2) of the bottom-left and top-right corners of the virtual overscan region, respectively.

For the BIASSECTAn and BIASSECTBn values, we want to use the six columns of prescan nearest the exposed area of the CCD. For subarray data, as there is no virtual overscan, the VXn and VYn values will always be 0.

In [ ]:
chip = 1 if amp in ['A', 'B'] else 2

# Identify the six (or less) prescan columns nearest the exposed area.
bias_a = [0, 0]
if leading > 0:
    bias_a[1] = leading
    bias_a[0] = leading-5 if leading > 5 else 0
        
bias_b = [0, 0]
if trailing > 0:
    bias_b[0] = xsize+1
    bias_b[1] = xsize+5 if trailing > 5 else xsize+trailing

# Construct our new definition.
custom_definition = (amp, chip, 1, 1, xsize, ysize, leading, trailing, 0, 0, 
                     bias_a[0], bias_a[1], bias_b[0], bias_b[1], 'Custom OSCN', 0, 0, 0, 0)

Now we need to open the custom `OSCTNAB` file and update the table to have our new definition. We will also need to update the raw FITS file to point to the new custom `OSCNTAB` reference file.

In [ ]:
with fits.open('my_oscntab.fits', mode='update') as hdu:
    old_table = hdu[1].data
    new_table = fits.FITS_rec.from_columns(old_table.columns, nrows=1)
    new_table[0] = custom_definition
    hdu[1].data = new_table
    
with fits.open('j92sa0w6q_raw.fits', mode='update') as hdu:
    hdu[0].header['OSCNTAB'] = 'my_oscntab.fits'

Finally, we print the table and check that we have entered it correctly. For the example dataset, we see that we have an 1180 x 1200 pixel image read out with amplifier A and containing 16 columns of physical prescan. During the calibration, columns 11-16 will be used to estimate the bias level of each row during the BLEVCORR step in `calacs`.

In [ ]:
Table(new_table)